<a href="https://colab.research.google.com/github/rayeeed/UFP_Project/blob/main/UFP_Prediction_datagen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
#from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score,cross_val_predict
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeCV
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectFromModel
from time import time
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import seaborn as sns
from sklearn.preprocessing import Normalizer,MinMaxScaler,StandardScaler,RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso, BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import RFE,RFECV
from sklearn.linear_model import (HuberRegressor,
                              	RANSACRegressor, TheilSenRegressor)

In [ ]:
!pip install optuna
import optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.5 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import KBinsDiscretizer

# Load the dataset
df = pd.read_excel('/content/drive/MyDrive/datasets_Rayeed_GSV_POI_LU/model_35.xlsx', index_col='site_name')

# Drop unnecessary columns
df.drop(['Site_type', 'OID'], inplace=True, axis=1)

# Separate the target variable and features
y = df['Mean PNC (# / cm3)']
X = df.drop(['Mean PNC (# / cm3)', 'Mean PM2.5 (µg/m3)'], axis=1)

In [ ]:
# Discretize the target variable into bins using 'quantile' strategy
binner = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
y_binned = binner.fit_transform(y.values.reshape(-1, 1)).flatten()

# Perform stratified train-test split
split = StratifiedShuffleSplit(n_splits=1, test_size=.2, random_state=42)
for train_index, test_index in split.split(X, y_binned):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Check the distribution of bins in the train and test sets
print("Train bin distribution:\n", pd.value_counts(y_binned[train_index], sort=False))
print("Test bin distribution:\n", pd.value_counts(y_binned[test_index], sort=False))

# Output the resulting dataframes
print("X_train:\n", X_train.head())
print("y_train:\n", y_train.head())
print("X_test:\n", X_test.head())
print("y_test:\n", y_test.head())

<ipython-input-4-a1486702b7a8>:12: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  print("Train bin distribution:\n", pd.value_counts(y_binned[train_index], sort=False))
<ipython-input-4-a1486702b7a8>:13: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  print("Test bin distribution:\n", pd.value_counts(y_binned[test_index], sort=False))


Train bin distribution:
 0.0    5
1.0    6
3.0    5
4.0    6
2.0    6
Name: count, dtype: int64
Test bin distribution:
 0.0    2
1.0    1
3.0    2
2.0    1
4.0    1
Name: count, dtype: int64
X_train:
                  gsv_wall_p100  gsv_building_p100  gsv_house_p100  \
site_name                                                           
Uttara_Sector1        6.797307          32.993471        0.232041   
Azimpur               0.000000           0.000000        0.000000   
US_Embassy            3.237500           2.430000        0.000000   
Elephant_road         0.070000          42.007500        0.000000   
Square_hospital      59.247500           0.000000        0.000000   

                 gsv_awning_p100  gsv_sky_p100  gsv_earth_p100  gsv_tree_p100  \
site_name                                                                       
Uttara_Sector1           0.09694     19.991354         3.48166      15.913745   
Azimpur                  0.00000      0.000000         0.00000       0.0

In [ ]:
from scipy.stats import ks_2samp
ks_statistic, p_value = ks_2samp(y_train, y_test)

print(f"K-S statistic: {ks_statistic}")
print(f"P-value: {p_value}")

# Interpretation
alpha = 0.05
if p_value < alpha:
    print("The null hypothesis is rejected. The distributions of the train and test sets are different.")
else:
    print("The null hypothesis cannot be rejected. The distributions of the train and test sets are the same.")

K-S statistic: 0.25
P-value: 0.842454033893869
The null hypothesis cannot be rejected. The distributions of the train and test sets are the same.


In [ ]:
# Apply Kolmogorov-Smirnov test for each feature
ks_results = {}
failed_features = []
alpha = 0.05
for feature in X.columns:
    ks_statistic, p_value = ks_2samp(X_train[feature], X_test[feature])
    ks_results[feature] = {'ks_statistic': ks_statistic, 'p_value': p_value}
    if p_value < alpha:
        failed_features.append(feature)

# Print K-S test results for each feature
for feature, result in ks_results.items():
    print(f"Feature: {feature}")
    print(f"K-S statistic: {result['ks_statistic']}")
    print(f"P-value: {result['p_value']}")
    if result['p_value'] < alpha:
        print("The null hypothesis is rejected. The distributions of the train and test sets are different for this feature.\n")
    else:
        print("The null hypothesis cannot be rejected. The distributions of the train and test sets are the same for this feature.\n")

# Print features that failed the K-S test
if failed_features:
    print("Features that failed the K-S test (null hypothesis rejected):")
    for feature in failed_features:
        print(f"- {feature}")
else:
    print("All features passed the K-S test (null hypothesis not rejected).")

Feature: gsv_wall_p100
K-S statistic: 0.32142857142857145
P-value: 0.5556032252116138
The null hypothesis cannot be rejected. The distributions of the train and test sets are the same for this feature.

Feature: gsv_building_p100
K-S statistic: 0.17857142857142858
P-value: 0.9883820703931286
The null hypothesis cannot be rejected. The distributions of the train and test sets are the same for this feature.

Feature: gsv_house_p100
K-S statistic: 0.10714285714285714
P-value: 0.9999998512905011
The null hypothesis cannot be rejected. The distributions of the train and test sets are the same for this feature.

Feature: gsv_awning_p100
K-S statistic: 0.21428571428571427
P-value: 0.9411846496106786
The null hypothesis cannot be rejected. The distributions of the train and test sets are the same for this feature.

Feature: gsv_sky_p100
K-S statistic: 0.2857142857142857
P-value: 0.7050303962215889
The null hypothesis cannot be rejected. The distributions of the train and test sets are the same

In [ ]:
X_train_dropped = X_train.drop(columns=failed_features)
X_test_dropped = X_test.drop(columns=failed_features)

In [ ]:
trans = StandardScaler()
X_st = trans.fit_transform(X_train_dropped)
X_st = pd.DataFrame(X_st, columns=X_train_dropped.columns, index=X_train_dropped.index)

In [ ]:
X_test_st = trans.transform(X_test_dropped)
X_test_st = pd.DataFrame(X_test_st, columns=X_test_dropped.columns, index=X_test_dropped.index)

In [ ]:
X_Pred = pd.read_excel('/content/drive/MyDrive/datasets_Rayeed_GSV_POI_LU/combined_total_v4.0.xlsx', index_col='OID_')

In [ ]:
col=X_st.columns
X_Pred_f1 = X_Pred[col]
X_Pred_f2= X_Pred_f1

In [ ]:
x_pred_st= trans.transform(X_Pred_f2)
x_pred_st = pd.DataFrame(x_pred_st, columns = X_Pred_f2.columns,index=X_Pred_f2.index)

# **SVR**

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_predict


# Feature set to be used for prediction
features = ['RdAll_150', 'gsv_house_p750', 'RdclP_150', 'gsv_plant_p250']

# Function to run the optimization and evaluate the model
def evaluate_model(features):
    X_Train = X_st[features]
    X_test = X_test_st[features]

    def objective(trial):
        C = trial.suggest_float("C", 0.01, 1000000.0, log=True)
        epsilon = trial.suggest_float("epsilon", 0.0001, 1000.0)
        gamma = trial.suggest_float("gamma", 0.0001, 100000, log=True)

        svr = SVR(C=C, epsilon=epsilon, gamma=gamma)
        predicted = cross_val_predict(svr, X_Train, y_train, cv=10)
        return mean_squared_error(y_train, predicted, squared=False)

    study = optuna.create_study(direction='minimize', study_name="SVR_RMSE")
    study.optimize(objective, n_trials=500)  # Run for 500 iterations

    best_params = study.best_params
    model = SVR(C=best_params["C"], epsilon=best_params["epsilon"], gamma=best_params["gamma"])
    model.fit(X_Train, y_train)

    pred_train = model.predict(X_Train)
    rmse_train = mean_squared_error(y_train, pred_train, squared=False)
    r2_train = r2_score(y_train, pred_train)

    predicted = cross_val_predict(model, X_Train, y_train, cv=10)
    rmse_cv = mean_squared_error(y_train, predicted, squared=False)
    r2_cv = r2_score(y_train, predicted)

    predicted_test = model.predict(X_test)
    rmse_test = mean_squared_error(y_test, predicted_test, squared=False)
    r2_test = r2_score(y_test, predicted_test)

    return {
        "features": features,
        "model": model,
        "rmse_train": rmse_train,
        "r2_train": r2_train,
        "rmse_cv": rmse_cv,
        "r2_cv": r2_cv,
        "rmse_test": rmse_test,
        "r2_test": r2_test
    }

# Evaluate the model using the specified feature set
print(f"Evaluating feature set: {features}")
try:
    result = evaluate_model(features)
    results = [result]
except ValueError as e:
    if "Input contains NaN" in str(e):
        print(f"Skipping evaluation due to NaN values in feature set: {features}")
    else:
        raise e  # Raise the error if it's not due to NaN values

# Get the trained model
model = results[0]['model']

# Generate predictions for the new dataset
x_pred_st_subset = x_pred_st[features]
predictions = model.predict(x_pred_st_subset)

# Save the predictions to a CSV file
predictions_df = pd.DataFrame(predictions, index=x_pred_st.index, columns=["Predictions"])
predictions_df.to_csv("/content/drive/MyDrive/datasets_Rayeed_GSV_POI_LU/predictions_ufp_svr.csv")

# Display statistics for the predictions
stats = predictions_df.describe()
print(stats)

# Save the statistics to a CSV file
stats.to_csv("/content/drive/MyDrive/datasets_Rayeed_GSV_POI_LU/predictions_ufp_svr_stats.csv")

print(predictions_df.head())
print(stats)
print(results)


[I 2024-08-12 06:53:16,051] A new study created in memory with name: SVR_RMSE
[I 2024-08-12 06:53:16,098] Trial 0 finished with value: 18818.107005708676 and parameters: {'C': 87.24039069530835, 'epsilon': 1.6299028590836198, 'gamma': 0.0002526127367742814}. Best is trial 0 with value: 18818.107005708676.
[I 2024-08-12 06:53:16,135] Trial 1 finished with value: 15620.131212790413 and parameters: {'C': 644457.3940418232, 'epsilon': 4.904509134954242, 'gamma': 0.8021042966133788}. Best is trial 1 with value: 15620.131212790413.
[I 2024-08-12 06:53:16,174] Trial 2 finished with value: 14605.842601471035 and parameters: {'C': 172776.35055175575, 'epsilon': 7.997391203980217, 'gamma': 0.08677194167601084}. Best is trial 2 with value: 14605.842601471035.
[I 2024-08-12 06:53:16,209] Trial 3 finished with value: 18798.043968384394 and parameters: {'C': 671.6266890171499, 'epsilon': 8.905770986006898, 'gamma': 3974.4775681520227}. Best is trial 2 with value: 14605.842601471035.
[I 2024-08-12 06

Evaluating feature set: ['RdAll_150', 'gsv_house_p750', 'RdclP_150', 'gsv_plant_p250']


[I 2024-08-12 06:53:16,288] Trial 5 finished with value: 13323.53380600527 and parameters: {'C': 19362.552413786016, 'epsilon': 1.4067515122031995, 'gamma': 0.07674946544688323}. Best is trial 5 with value: 13323.53380600527.
[I 2024-08-12 06:53:16,333] Trial 6 finished with value: 18818.73038111919 and parameters: {'C': 0.25616733787664797, 'epsilon': 4.952479557715871, 'gamma': 1.4108970997344508}. Best is trial 5 with value: 13323.53380600527.
[I 2024-08-12 06:53:16,379] Trial 7 finished with value: 18818.735901849213 and parameters: {'C': 14.834903297868921, 'epsilon': 3.587621139239451, 'gamma': 0.00013244919688520682}. Best is trial 5 with value: 13323.53380600527.
[I 2024-08-12 06:53:16,416] Trial 8 finished with value: 18818.797593288018 and parameters: {'C': 1.1651901946366587, 'epsilon': 9.001787235901515, 'gamma': 36548.87355731639}. Best is trial 5 with value: 13323.53380600527.
[I 2024-08-12 06:53:16,456] Trial 9 finished with value: 18818.797593288018 and parameters: {'C'

         Predictions
count    3724.000000
mean    59236.255412
std     16297.364540
min     15212.293226
25%     47166.194261
50%     56498.360836
75%     68562.704849
max    115205.211164
       Predictions
OID_              
1268  96185.485784
1271  94583.035187
1272  82623.666330
1273  91613.139199
1274  86737.209971
         Predictions
count    3724.000000
mean    59236.255412
std     16297.364540
min     15212.293226
25%     47166.194261
50%     56498.360836
75%     68562.704849
max    115205.211164
[{'features': ['RdAll_150', 'gsv_house_p750', 'RdclP_150', 'gsv_plant_p250'], 'model': SVR(C=994619.509641855, epsilon=9.495431518195927, gamma=0.004148820832977478), 'rmse_train': 9776.109789149292, 'r2_train': 0.7026120075317565, 'rmse_cv': 10700.497435069245, 'r2_cv': 0.6437136178175684, 'rmse_test': 16901.635384260142, 'r2_test': 0.5247182571259927}]


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_predict


# Feature set to be used for prediction
features = ['RdAll_150', 'gsv_house_p750', 'RdclP_150', 'gsv_plant_p250']

# Function to run the optimization and evaluate the model
def evaluate_model(features):
    X_Train = X_st[features]
    X_test = X_test_st[features]

    def objective(trial):
        C = trial.suggest_float("C", 0.01, 1000000.0, log=True)
        epsilon = trial.suggest_float("epsilon", 0.0001, 1000.0)
        gamma = trial.suggest_float("gamma", 0.0001, 100000, log=True)

        svr = SVR(C=C, epsilon=epsilon, gamma=gamma)
        predicted = cross_val_predict(svr, X_Train, y_train, cv=10)
        return mean_squared_error(y_train, predicted, squared=False)

    study = optuna.create_study(direction='minimize', study_name="SVR_RMSE")
    study.optimize(objective, n_trials=500)  # Run for 500 iterations

    best_params = study.best_params
    model = SVR(C=best_params["C"], epsilon=best_params["epsilon"], gamma=best_params["gamma"])
    model.fit(X_Train, y_train)

    pred_train = model.predict(X_Train)
    rmse_train = mean_squared_error(y_train, pred_train, squared=False)
    r2_train = r2_score(y_train, pred_train)

    predicted = cross_val_predict(model, X_Train, y_train, cv=10)
    rmse_cv = mean_squared_error(y_train, predicted, squared=False)
    r2_cv = r2_score(y_train, predicted)

    predicted_test = model.predict(X_test)
    rmse_test = mean_squared_error(y_test, predicted_test, squared=False)
    r2_test = r2_score(y_test, predicted_test)

    return {
        "features": features,
        "model": model,
        "rmse_train": rmse_train,
        "r2_train": r2_train,
        "rmse_cv": rmse_cv,
        "r2_cv": r2_cv,
        "rmse_test": rmse_test,
        "r2_test": r2_test
    }

# Evaluate the model using the specified feature set
print(f"Evaluating feature set: {features}")
try:
    result = evaluate_model(features)
    results = [result]
except ValueError as e:
    if "Input contains NaN" in str(e):
        print(f"Skipping evaluation due to NaN values in feature set: {features}")
    else:
        raise e  # Raise the error if it's not due to NaN values

# Get the trained model
model = results[0]['model']

# Generate predictions for the new dataset
x_pred_st_subset = x_pred_st[features]
predictions = model.predict(x_pred_st_subset)

# Save the predictions to a CSV file
predictions_df = pd.DataFrame(predictions, index=x_pred_st.index, columns=["Predictions"])
predictions_df.to_csv("/content/drive/MyDrive/datasets_Rayeed_GSV_POI_LU/predictions_ufp_svr.csv")

# Display statistics for the predictions
stats = predictions_df.describe()
print(stats)

# Save the statistics to a CSV file
stats.to_csv("/content/drive/MyDrive/datasets_Rayeed_GSV_POI_LU/predictions_ufp_svr_stats.csv")

print(predictions_df.head())
print(stats)
print(results)


[I 2024-08-20 06:48:03,802] A new study created in memory with name: SVR_RMSE
[I 2024-08-20 06:48:03,885] Trial 0 finished with value: 18775.81538244699 and parameters: {'C': 80.17618238610322, 'epsilon': 855.6900257309021, 'gamma': 46.68842297267074}. Best is trial 0 with value: 18775.81538244699.
[I 2024-08-20 06:48:03,970] Trial 1 finished with value: 11877.57644197543 and parameters: {'C': 120674.57904440945, 'epsilon': 982.3174269814334, 'gamma': 0.03590967695742012}. Best is trial 1 with value: 11877.57644197543.


Evaluating feature set: ['RdAll_150', 'gsv_house_p750', 'RdclP_150', 'gsv_plant_p250']


[I 2024-08-20 06:48:04,040] Trial 2 finished with value: 18919.121048617344 and parameters: {'C': 183427.07377337667, 'epsilon': 387.8885076186362, 'gamma': 83.40685302493476}. Best is trial 1 with value: 11877.57644197543.
[I 2024-08-20 06:48:04,098] Trial 3 finished with value: 18776.46961871732 and parameters: {'C': 0.17801532651876562, 'epsilon': 829.5602942572227, 'gamma': 0.11601085875180718}. Best is trial 1 with value: 11877.57644197543.
[I 2024-08-20 06:48:04,161] Trial 4 finished with value: 15435.959001906751 and parameters: {'C': 3415.2549220868814, 'epsilon': 679.9455765681425, 'gamma': 0.09194895193958577}. Best is trial 1 with value: 11877.57644197543.
[I 2024-08-20 06:48:04,228] Trial 5 finished with value: 18777.150569535537 and parameters: {'C': 0.2140180518390079, 'epsilon': 785.7367281269226, 'gamma': 0.01717958612035459}. Best is trial 1 with value: 11877.57644197543.
[I 2024-08-20 06:48:04,289] Trial 6 finished with value: 18777.611661695195 and parameters: {'C': 

         Predictions
count    3724.000000
mean    59258.547312
std     16382.418807
min     15188.011266
25%     47112.920756
50%     56456.905916
75%     68672.918139
max    115644.531824
       Predictions
OID_              
1268  97276.002403
1271  95855.602245
1272  82620.526285
1273  92205.172739
1274  88097.471533
         Predictions
count    3724.000000
mean    59258.547312
std     16382.418807
min     15188.011266
25%     47112.920756
50%     56456.905916
75%     68672.918139
max    115644.531824
[{'features': ['RdAll_150', 'gsv_house_p750', 'RdclP_150', 'gsv_plant_p250'], 'model': SVR(C=984839.7383362645, epsilon=73.37370163849863, gamma=0.004035060367827422), 'rmse_train': 9782.491289105483, 'r2_train': 0.7022236320113742, 'rmse_cv': 10700.744483401128, 'r2_cv': 0.6436971660641789, 'rmse_test': 16895.34169614167, 'r2_test': 0.5250721540190683}]


In [ ]:
results

[{'features': ['RdAll_150', 'gsv_house_p750', 'RdclP_150', 'gsv_plant_p250'],
  'model': SVR(C=991985.2774618675, epsilon=2.739518480154803, gamma=0.0041511012624511935),
  'rmse_train': 9777.565110466385,
  'r2_train': 0.7025234595697842,
  'rmse_cv': 10701.800035203421,
  'r2_cv': 0.6436268691519033,
  'rmse_test': 16892.822348861948,
  'r2_test': 0.5252137811006332}]

# **Linear regression**

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_predict


# Feature set to be used for prediction
features = ['RdAll_150', 'gsv_house_p750', 'RdL3_1000', 'RdclP_150', 'gsv_plant_p250']

# Function to run the optimization and evaluate the model
def evaluate_model(features):
    X_Train = X_st[features]
    X_test = X_test_st[features]

    def objective(trial):
        # No hyperparameters to optimize for LinearRegression
        model = LinearRegression()
        predicted = cross_val_predict(model, X_Train, y_train, cv=10)
        return mean_squared_error(y_train, predicted, squared=False)

    study = optuna.create_study(direction='minimize', study_name="LinearRegression_RMSE")
    study.optimize(objective, n_trials=1)  # Only run 1 trial since there are no hyperparameters

    model = LinearRegression()
    model.fit(X_Train, y_train)

    pred_train = model.predict(X_Train)
    rmse_train = mean_squared_error(y_train, pred_train, squared=False)
    r2_train = r2_score(y_train, pred_train)

    predicted = cross_val_predict(model, X_Train, y_train, cv=10)
    rmse_cv = mean_squared_error(y_train, predicted, squared=False)
    r2_cv = r2_score(y_train, predicted)

    predicted_test = model.predict(X_test)
    rmse_test = mean_squared_error(y_test, predicted_test, squared=False)
    r2_test = r2_score(y_test, predicted_test)

    return {
        "features": features,
        "model": model,
        "rmse_train": rmse_train,
        "r2_train": r2_train,
        "rmse_cv": rmse_cv,
        "r2_cv": r2_cv,
        "rmse_test": rmse_test,
        "r2_test": r2_test
    }

# Evaluate the model using the specified feature set
print(f"Evaluating feature set: {features}")
try:
    result = evaluate_model(features)
    results = [result]
except ValueError as e:
    if "Input contains NaN" in str(e):
        print(f"Skipping evaluation due to NaN values in feature set: {features}")
    else:
        raise e  # Raise the error if it's not due to NaN values

# Get the trained model
model = results[0]['model']

# Generate predictions for the new dataset
x_pred_st_subset = x_pred_st[features]
predictions = model.predict(x_pred_st_subset)

# Save the predictions to a CSV file
predictions_df = pd.DataFrame(predictions, index=x_pred_st.index, columns=["Predictions"])
predictions_df.to_csv("/content/drive/MyDrive/datasets_Rayeed_GSV_POI_LU/predictions_UFP_linear_regression.csv")

# Display statistics for the predictions
stats = predictions_df.describe()
print(stats)

# Save the statistics to a CSV file
stats.to_csv("/content/drive/MyDrive/datasets_Rayeed_GSV_POI_LU/predictions_UFP_linear_regression_stats.csv")

print(predictions_df.head())
print(stats)
print(results)

[I 2024-08-20 03:04:40,096] A new study created in memory with name: LinearRegression_RMSE
[I 2024-08-20 03:04:40,151] Trial 0 finished with value: 12161.131717562397 and parameters: {}. Best is trial 0 with value: 12161.131717562397.


Evaluating feature set: ['RdAll_150', 'gsv_house_p750', 'RdL3_1000', 'RdclP_150', 'gsv_plant_p250']
         Predictions
count    3724.000000
mean    62486.515653
std     30444.419026
min      1119.961646
25%     45582.405346
50%     55158.728015
75%     68962.794743
max    616519.874756
        Predictions
OID_               
1268  149455.978817
1271  158056.931113
1272   91228.683478
1273  126022.891351
1274  164691.625453
         Predictions
count    3724.000000
mean    62486.515653
std     30444.419026
min      1119.961646
25%     45582.405346
50%     55158.728015
75%     68962.794743
max    616519.874756
[{'features': ['RdAll_150', 'gsv_house_p750', 'RdL3_1000', 'RdclP_150', 'gsv_plant_p250'], 'model': LinearRegression(), 'rmse_train': 9360.18430862223, 'r2_train': 0.7273785084555218, 'rmse_cv': 12161.131717562397, 'r2_cv': 0.5398077786621378, 'rmse_test': 15936.174772649556, 'r2_test': 0.5774658229816017}]


# **XGBoost**

In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_predict
from xgboost import XGBRegressor
import optuna

# Feature set to be used for prediction
features = ['RdAll_150', 'gsv_house_p750', 'RdL3_750', 'gsv_plant_p250']

# Function to run the optimization and evaluate the model
def evaluate_model(features):
    X_Train = X_st[features]
    X_test = X_test_st[features]

    def objective(trial):
        #gamma = trial.suggest_float('gamma', 0, 4000)  # Uncommented to include in optimization
        learning_rate = trial.suggest_float('learning_rate', 0.0005, 1.5)
        #max_depth = int(trial.suggest_int('max_depth', 50, 5000))  # Uncommented to include in optimization
        #min_child_weight = int(trial.suggest_int('min_child_weight', 0, 2000))  # Uncommented to include in optimization
        n_estimators = int(trial.suggest_int('n_estimators', 50, 10000))
        reg_alpha = trial.suggest_float('reg_alpha', 0.001, 1000)
        reg_lambda = trial.suggest_float('reg_lambda', 0.001, 30)
        #subsample = trial.suggest_float('subsample', 0.6, 1)  # Uncommented to include in optimization

        XGB = XGBRegressor(
            booster='gblinear',
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            objective="reg:squarederror",
            seed=42
        )

        predicted = cross_val_predict(XGB, X_Train, y_train, cv=10)
        return mean_squared_error(y_train, predicted, squared=False)

    study = optuna.create_study(direction='minimize', study_name="XGBRegressor_RMSE")
    study.optimize(objective, n_trials=500)  # Run for 500 iterations

    best_params = study.best_params
    model = XGBRegressor(
        booster='gblinear',
        n_estimators=best_params['n_estimators'],
        learning_rate=best_params['learning_rate'],
        reg_alpha=best_params['reg_alpha'],
        reg_lambda=best_params['reg_lambda'],
        objective="reg:squarederror",
        seed=42
    )
    model.fit(X_Train, y_train)

    pred_train = model.predict(X_Train)
    rmse_train = mean_squared_error(y_train, pred_train, squared=False)
    r2_train = r2_score(y_train, pred_train)

    predicted = cross_val_predict(model, X_Train, y_train, cv=10)
    rmse_cv = mean_squared_error(y_train, predicted, squared=False)
    r2_cv = r2_score(y_train, predicted)

    predicted_test = model.predict(X_test)
    rmse_test = mean_squared_error(y_test, predicted_test, squared=False)
    r2_test = r2_score(y_test, predicted_test)

    return {
        "features": features,
        "model": model,
        "rmse_train": rmse_train,
        "r2_train": r2_train,
        "rmse_cv": rmse_cv,
        "r2_cv": r2_cv,
        "rmse_test": rmse_test,
        "r2_test": r2_test
    }

# Evaluate the model using the specified feature set
print(f"Evaluating feature set: {features}")
try:
    result = evaluate_model(features)
    results = [result]
except ValueError as e:
    if "Input contains NaN" in str(e):
        print(f"Skipping evaluation due to NaN values in feature set: {features}")
    else:
        raise e  # Raise the error if it's not due to NaN values

# Get the trained model
model = results[0]['model']

# Generate predictions for the new dataset
x_pred_st_subset = x_pred_st[features]
predictions = model.predict(x_pred_st_subset)

# Save the predictions to a CSV file
predictions_df = pd.DataFrame(predictions, index=x_pred_st.index, columns=["Predictions"])
predictions_df.to_csv("/content/drive/MyDrive/datasets_Rayeed_GSV_POI_LU/predictions_UFP_xgb.csv")

# Display statistics for the predictions
stats = predictions_df.describe()
print(stats)

# Save the statistics to a CSV file
stats.to_csv("/content/drive/MyDrive/datasets_Rayeed_GSV_POI_LU/predictions_pm25_UFP_stats.csv")

print(predictions_df.head())
print(stats)
print(results)


[I 2024-08-20 03:28:37,461] A new study created in memory with name: XGBRegressor_RMSE


Evaluating feature set: ['RdAll_150', 'gsv_house_p750', 'RdL3_750', 'gsv_plant_p250']


[I 2024-08-20 03:28:42,940] Trial 0 finished with value: 18384.4117526309 and parameters: {'learning_rate': 0.9706807220153558, 'n_estimators': 3088, 'reg_alpha': 752.8120721072244, 'reg_lambda': 29.18816465608563}. Best is trial 0 with value: 18384.4117526309.
[I 2024-08-20 03:28:46,822] Trial 1 finished with value: 18121.352838848998 and parameters: {'learning_rate': 1.039073180320859, 'n_estimators': 6825, 'reg_alpha': 835.5622661560552, 'reg_lambda': 18.751566200340072}. Best is trial 1 with value: 18121.352838848998.
[I 2024-08-20 03:28:53,532] Trial 2 finished with value: 18064.31677228967 and parameters: {'learning_rate': 0.16581504636854955, 'n_estimators': 6263, 'reg_alpha': 611.8723603639182, 'reg_lambda': 17.94988433176795}. Best is trial 2 with value: 18064.31677228967.
[I 2024-08-20 03:28:55,180] Trial 3 finished with value: 15555.535503836241 and parameters: {'learning_rate': 0.4814343807989636, 'n_estimators': 2645, 'reg_alpha': 404.9329144392864, 'reg_lambda': 3.3599341

         Predictions
count    3724.000000
mean    60851.964844
std     25926.449219
min     20543.363281
25%     46073.595703
50%     54468.962891
75%     66430.283203
max    531730.875000
        Predictions
OID_               
1268  134239.250000
1271  141567.765625
1272   85380.656250
1273  115569.367188
1274  148891.703125
         Predictions
count    3724.000000
mean    60851.964844
std     25926.449219
min     20543.363281
25%     46073.595703
50%     54468.962891
75%     66430.283203
max    531730.875000
[{'features': ['RdAll_150', 'gsv_house_p750', 'RdL3_750', 'gsv_plant_p250'], 'model': XGBRegressor(base_score=None, booster='gblinear', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning